In [1]:
import sys
sys.path.append('../_amigocloud')

from werkzeug.wrappers import Request, Response
#from flask import Flask, render_template
from jinja2 import Template

#from datetime import date
from datetime import datetime, timedelta

import collections
from amigocloud import AmigoCloud

from docxtpl import DocxTemplate
import docxtpl
from docx.shared import Mm
import requests

#from docx2pdf import convert
import os

import pandas as pd
import json

In [2]:
# ruta de carpera donde estan las llaves
ruta = open('../ruta.txt')
ruta = ruta.readlines()[0]

In [3]:
# leer token
acceso = open(ruta + '/_keys/api_amigocloud.key','r')
api_token = acceso.readlines()[0]
acceso.close()
amigocloud = AmigoCloud(token=api_token)
amigocloud

In [4]:
fecha = '2024-05-30'

In [5]:
query_insp = {'query': 'select id, canhero from dataset_291595 where date(fecha_registro)=\'{fech}\''.format(fech=fecha)}
project = 'https://app.amigocloud.com/api/v1/projects/31805/sql'
select_insp = amigocloud.get(project, query_insp)
data_insp = select_insp['data']
data_insp

[{'id': 166, 'canhero': '794 / AGUILERA WENDE MARIANO'},
 {'id': 167, 'canhero': '794 / AGUILERA WENDE MARIANO'},
 {'id': 165, 'canhero': '794 / AGUILERA WENDE MARIANO'},
 {'id': 164, 'canhero': '794 / AGUILERA WENDE MARIANO'}]

In [6]:
data_insp = [{'id': 154, 'canhero': '2794 / DELGADILLO RODRIGUEZ JUAN CARLOS'}]

In [7]:
for j in data_insp:
    id_insp = j['id']
    print('Generando reporte de:', j['canhero'], '................................................')
    # SELECT DATOS DE LAS INPECCION
    query = {'query': 'select * from dataset_291595 where id={id}'.format(id=id_insp)}
    project = 'https://app.amigocloud.com/api/v1/projects/31805/sql'
    select = amigocloud.get(project, query)
    data = select['data'][0]
    # convertir fecha a formato dd/mm/aaaa
    data['fecha_registro'] = datetime.strptime(data['fecha_registro'][0:10], '%Y-%m-%d').strftime('%d/%m/%Y')
    # convertir la insp en fecha
    insp = collections.namedtuple("insp", data.keys())(*data.values())
    
    # DATOS DE MUESTRAS
    query = {'query': 'select * from dataset_291604 where calidad_ref_muestra=\'{amigo_id}\''.format(amigo_id=insp.amigo_id)}
    project = 'https://app.amigocloud.com/api/v1/projects/31805/sql'
    select = amigocloud.get(project, query)
    data = select['data']
    # se recorre las muestras para crear un obj y agregarlo a una lista
    muestras =[]
    for i in data:
        # se conviente a entrero
        i['arranquio_de_cepa'] = int(i['arranquio_de_cepa'])
        muestra = collections.namedtuple("muestra", i.keys())(*i.values())
        muestras.append(muestra)
    
    # CALCUALR PROMEDIO DE PESOS
    keys_to_average = ['astillas', 'tocon', 'punta', 'canha_corta', 'canha_larga', 'arranquio_de_cepa', 'estimacion_promedio']
    promedio = {}
    for key in keys_to_average:
        total = sum([item[key] for item in data])
        average = total / len(data)
        promedio[key] = average
    promedio['perdida_total'] = promedio['astillas'] + promedio['tocon'] + promedio['punta'] + promedio['canha_corta'] + promedio['canha_larga']
    promedio['perdida_porcen'] = (promedio['perdida_total'] / promedio['estimacion_promedio'])*100
    promedio['arranquio_de_cepa'] = int(promedio['arranquio_de_cepa'])
    promedio_muestras = collections.namedtuple("promedio", promedio.keys())(*promedio.values())
    
    # SE EXTRAE LOS DATOS DE LAS FOTOS EXISTENTES EN LAS MUESTAS
    fotos = []
    for i in muestras:
        query = {'query': 'select muestra.nombre_muestra, muestra.obs, gal.s3_filename from dataset_291604 muestra inner join gallery_41829 gal on muestra.amigo_id = gal.source_amigo_id where muestra.amigo_id=\'{amigo_id}\''.format(amigo_id=i.amigo_id)}
        project = 'https://app.amigocloud.com/api/v1/projects/31805/sql'
        select = amigocloud.get(project, query)
        if len(select['data'])>0:
            data = select['data'][0]
        else:
            continue
        #data = select['data'][0]
        fotos.append(data)
        print(data)
    
    # SE INSTANCIA AL TEMPLATE
    doc = DocxTemplate(ruta + "/templates/tpl_rpt_calidad-cosecha.docx")
    
    # SE DESCARGAN LAS FOTOS
    lista_fotos_inline = []
    for foto in fotos:
        response = requests.get("https://www-amigocloud.s3.amazonaws.com/gallery/" + foto['s3_filename'])
        file = open(ruta + '/fotos/' + foto['s3_filename'], "wb")
        file.write(response.content)
        file.close()
        lista_fotos_inline.append({
            'foto': docxtpl.InlineImage(doc, image_descriptor=ruta+'/fotos/' + foto['s3_filename'], width=Mm(60)), 
            'muestra': foto['nombre_muestra'],
            'obs':foto['obs']
        })
    
    # SE ESTABLESE LA FIRMA DEL TECNICO
    firma_respon = None
    if insp.responsable == 'Juan Pablo Jaldin':
        firma_respon = docxtpl.InlineImage(doc, image_descriptor=ruta+'/templates/firma_jaldin.png', width=Mm(60))
    else:
        firma_respon = docxtpl.InlineImage(doc, image_descriptor=ruta+'/templates/firma_rogelio.png', width=Mm(60))
    
    # NORMBRE DEL ARCHIVO
    filename = '/_' + insp.canhero.split(' / ')[0] + '_CDC_' + insp.fecha_registro.replace('/','-') + '_' + insp.canhero.split(' / ')[1] + '_' + str(insp.id) + '.docx'
    # CREAR EL CONTEXTO DEL DOC Y CREAR EL DOC
    context = {'insp':insp, 'muestras':muestras, 'promedio':promedio_muestras, 'fotos':lista_fotos_inline, 'firma':firma_respon}
    doc.render(context)
    doc.save(ruta + '/informes' + filename)
    print('Trarea terminada.')
print('END==============================================')

Generando reporte de: 794 / AGUILERA WENDE MARIANO ................................................
{'nombre_muestra': 'M1', 'obs': 'Perdida', 's3_filename': 'IMG-20240530-WA0024.jpg'}
{'nombre_muestra': 'M2', 'obs': 'Perdida', 's3_filename': 'IMG-20240530-WA0025.jpg'}
Trarea terminada.
Generando reporte de: 794 / AGUILERA WENDE MARIANO ................................................
{'nombre_muestra': 'M1', 'obs': 'Perdida', 's3_filename': 'IMG-20240530-WA0026.jpg'}
{'nombre_muestra': 'M2', 'obs': 'Pardida', 's3_filename': 'IMG-20240530-WA0032.jpg'}
Trarea terminada.
Generando reporte de: 794 / AGUILERA WENDE MARIANO ................................................
{'nombre_muestra': 'M2', 'obs': 'Perdida', 's3_filename': 'IMG-20240530-WA0034.jpg'}
{'nombre_muestra': 'M1', 'obs': 'Perdida', 's3_filename': 'IMG-20240530-WA0033.jpg'}
Trarea terminada.
Generando reporte de: 794 / AGUILERA WENDE MARIANO ................................................
{'nombre_muestra': 'M1', 'obs': 'Poc

## Convertir a PDF

In [8]:
path = r'' + ruta + '/informes'
contenido = os.listdir(path)
rutas =[]
i = 0
for name in contenido:
    rutas.append(path + '/' + name)
    i+=1

print('SE ENCONTRARON ' + str(i) + ' INFORMES')
print(len(rutas), 'INFORMES')
rutas

SE ENCONTRARON 1 INFORMES
1 INFORMES


['D:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes/5406_CDC_20-06-2023_HURTADO MENDEZ RONALD_44.docx']

In [9]:
i = 1
for filename in rutas:
    print('CONVIRTIENDO... INFORME ' + str(i) + ' ' + filename)
    # se realiza la convrsion pasando parametro de salida y entrada, 
    # para la salida se reemplaza la extencion y la ruta
    print(filename)
    print(filename.replace('.docx','.pdf').replace('informes','pdfs'))
    pdf = filename.replace('.docx','.pdf').replace('informes','pdfs').split('/')[-1]
    convert(filename, r'D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON\pdf\\' + pdf)
    i+=1
print('END')

CONVIRTIENDO... INFORME 1 D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/5406_CDC_20-06-2023_HURTADO MENDEZ RONALD_44.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/5406_CDC_20-06-2023_HURTADO MENDEZ RONALD_44.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/pdfs/5406_CDC_20-06-2023_HURTADO MENDEZ RONALD_44.pdf


  0%|          | 0/1 [00:00<?, ?it/s]

END


## Cargar a DRIVE